In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import time
import pandas as pd
from intermediates import *

In [2]:
import lsst.sims.maf.slicers as slicers
from lsst.obs.lsstSim import LsstSimMapper
from lsst.sims.utils import ObservationMetaData
from lsst.sims.coordUtils import chipNameFromRaDec

In [3]:
dbpath= '/Volumes/Transcend/minion_1016_sqlite.db'
#simdata= getSimData(dbpath, 'r')   # get the OpSim columns

In [6]:
def findChips(dpbath, focusDither, focusID, 
              filterBand= 'r', disc= True):
    
    FOV_radius= 0.0305
    nside= 512
    printProgress('Getting simData ... ', highlight= True)
    simdata= getSimData(dbpath, filterBand, ['expDate', 'obsHistID'])
    
    printProgress('Finding region pixels ... ', highlight= True)
    centralRA, centralDec, regionPixels= findRegionPixels(focusID, simdata,
                                                          nside, disc, FOV_radius)
    totPixels= len(regionPixels)
    printProgress('Total number of pixels: %f'%(totPixels))
                  
    # set up the slicer
    hpSlicer= slicers.HealpixSlicer(nside= nside)
    hpSlicer.setupSlicer(simdata)    # need to slice data?
    
    camera = LsstSimMapper().camera
    chipNames= []
    obsIDs= []
    expDates= []
    for p, pixel in enumerate(regionPixels):
        pixRA, pixDec= hpSlicer._pix2radec(pixel)    # radians returned    
        indObsInPixel = hpSlicer._sliceSimData(pixel)
        
        for index in indObsInPixel['idxs']:
            # get data
            # for identifying each visit
            expDate= np.degrees(simdata[index]['expDate'])
            obsID= np.degrees(simdata[index]['obsHistID'])
            
            # for chip finding
            if (focusDither=='NoDither'):
                pointingRA= np.degrees(simdata[index]['fieldRA'])
                pointingDec= np.degrees(simdata[index]['fieldDec'])
            else:
                pointingRA= np.degrees(simdata[index]['ditheredRA'])
                pointingDec= np.degrees(simdata[index]['ditheredDec'])
            rotSkyPos= np.degrees(simdata[index]['rotSkyPos'])
            expMJD= np.degrees(simdata[index]['expMJD'])
            
            # set up for the finding the chips
            obs = ObservationMetaData(pointingRA= pointingRA, pointingDec= pointingDec,
                                      rotSkyPos= rotSkyPos, mjd= expMJD)
            chipsInVisit= chipNameFromRaDec(np.degrees(pixRA), np.degrees(pixDec),
                                            camera=camera, obs_metadata=obs)
            if chipsInVisit is not None:
                obsIDs.append(obsID)
                expDates.append(expDate)
                chipNames.append(chipsInVisit)
        percentDone= 100.*(p+1)/totPixels
        
        if (percentDone%.5<0.02):
            printProgress('Percent pixels done: %f'%(percentDone))
        if (percentDone>1.):
            return [obsIDs, expDates, chipNames]

In [7]:
obsIDs, expDates, chipNames= findChips(dbpath, '', 1447, 
                                       filterBand= 'r', disc= True)


#############################################
## Getting simData ... 

#############################################
## Finding region pixels ... 
#
## Total number of pixels: 4568.000000
Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


/Users/Humna/lsst/DarwinX86/sims_utils/12.0-13-g6534730/python/lsst/sims/utils/ModifiedJulianDate.py:188: UTCtoUT1Warning: UTC is outside of IERS table for UT1-UTC.
Returning UT1 = UTC for lack of a better idea
This warning was caused by calling ModifiedJulianDate.dut1

  category=UTCtoUT1Warning)


#
## Percent pixels done: 0.503503
#
## Percent pixels done: 1.007005


In [9]:
pd.DataFrame([obsIDs, expDates, chipNames], ['obsHistID', 'expDate', 'Chips'])

,0,1,2,3,4,5,6,7,8,9,...,7684,7685,7686,7687,7688,7689,7690,7691,7692,7693
obsHistID,1.24258e+08,1.14245e+08,2.61799e+07,1.14247e+08,1.2506e+08,7.44423e+07,8.15184e+07,2.61783e+07,1.34065e+07,7.44439e+07,...,1.00948e+08,8.69288e+07,7.37602e+07,2.76327e+07,4.03156e+07,8.68346e+07,7.19673e+07,1.36545e+08,9.62753e+07,1.25175e+08
expDate,1.58178e+10,1.45795e+10,3.26843e+09,1.45796e+10,1.59168e+10,9.47024e+09,1.04219e+10,3.26836e+09,1.59534e+09,9.47031e+09,...,1.29755e+10,1.11291e+10,9.38649e+09,3.48078e+09,5.13466e+09,1.11192e+10,9.18839e+09,1.75507e+10,1.24113e+10,1.59316e+10
Chips,"R:2,1 S:1,2","R:0,1 S:1,0","R:2,0 S:1,1","R:1,0 S:0,1","R:0,3 S:1,1","R:0,1 S:2,0","R:1,4 S:2,1","R:2,0 S:1,1","R:2,3 S:0,2","R:0,1 S:2,0",...,"R:4,1 S:1,2","R:2,2 S:2,0","R:2,1 S:2,0","R:1,2 S:0,2","R:3,2 S:1,0","R:2,3 S:1,1","R:3,4 S:1,1","R:3,0 S:0,2","R:3,0 S:1,2","R:3,3 S:0,2"


In [10]:
len(chipNames)

7694

In [12]:
len(np.unique(expDates))

397